In [17]:
import tensorflow as tf
from transformers import BertTokenizer
from transformers import TFBertForSequenceClassification

from sklearn.model_selection import train_test_split

In [2]:
from src.dataset import load_dataset, Species, Modification

In [3]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [4]:
train_data = load_dataset(Species.human, Modification.psi)

In [19]:
x_train, x_test, y_train, y_test = train_test_split(
    train_data.samples,
    train_data.targets,
    test_size=0.2
)

In [26]:
x_train_encodings = tokenizer(
    [value[0] for value in x_train.sequence.values],
    truncation=True, padding=True, max_length=41
)

x_test_encodings = tokenizer(
    [value[0] for value in x_test.sequence.values],
    truncation=True, padding=True, max_length=41
)

In [33]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(x_train_encodings),
    y_train
))
test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(x_test_encodings),
    y_test
))

In [ ]:
model_bert = TFBertForSequenceClassification.from_pretrained('bert-base-cased')

In [34]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model_bert.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [ ]:
model_bert.fit(train_dataset.shuffle(100).batch(32), validation_data=test_dataset.batch(32), epochs=3, batch_size=32)

Epoch 1/3
198/198 [==============================] - 91s 350ms/step - loss: 0.7043 - accuracy: 0.5020 - val_loss: 0.6931 - val_accuracy: 0.4991
Epoch 2/3
 41/198 [=====>........................] - ETA: 49s - loss: 0.7012 - accuracy: 0.4931